In [104]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

import pymysql
import pandas as pd
pymysql.install_as_MySQLdb()

In [105]:
Base = declarative_base()

class Rider(Base):
    __tablename__ = 'rider'
    id = Column(Integer, primary_key=True)
    bike_id = Column(Integer)
    start_time = Column(Integer)
    start_station_id = Column(Integer)
    end_time = Column(Integer)
    end_station_id = Column(Integer)
    duration = Column(Integer)
    user_type = Column(Integer)
    member_age = Column(Integer)
    member_gender = Column(Integer)
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station_id = Column(Integer)
    station_name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    neighborhood = Column(String(255))
    zipcode = Column(Integer)


In [106]:
file = 'Data/stationsample.csv'
pd.read_csv(file).drop(columns=['Unnamed: 0']).to_csv('Data/stationsample_cleaned.csv', index=False)

In [107]:
cleanfile = 'Data/stationsample_cleaned.csv'
df = pd.read_csv(cleanfile)
df.head()

,station_id,station_name,latitude,longitude,neighborhood,zipcode
0,118,Eureka Valley Recreation Center,37.759177,-122.436943,The Castro,94114
1,141,Valencia St at Cesar Chavez St,37.747998,-122.420219,Bernal Heights,94110
2,36,Folsom St at 3rd St,37.783830,-122.398870,South of Market,94103
3,81,Berry St at 4th St,37.775880,-122.393170,China Basin,94107
4,95,Sanchez St at 15th St,37.766219,-122.431060,The Castro,94114


In [108]:
engine = create_engine('sqlite:///GoBike.sqlite')
conn = engine.connect()

In [109]:
Base.metadata.create_all(engine)

In [110]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [111]:
for index, row in df.iterrows():
    session.add(Station(station_id=row['station_id'],
                        station_name=row['station_name'], 
                        latitude=row['latitude'], 
                        longitude=row['longitude'], 
                        neighborhood=row['neighborhood'], 
                        zipcode=row['zipcode']))

In [112]:
gobike = 'Data/GoBikeRiderSample.csv'
pd.read_csv(gobike).drop(columns=['Unnamed: 0']).to_csv('Data/GoBikeRiderClean.csv', index=False)

In [113]:
file2 = 'Data/GoBikeRiderClean.csv'
df2 = pd.read_csv(file2)
df2.head()

,bike_id,start_time,start_station_id,end_time,end_station_id,duration_sec,user_type,member_age,member_gender
0,4036,1527836548,118,1527837055,141,507,0,46,0
1,1607,1527836557,36,1527836841,81,283,0,31,0
2,1701,1527836558,95,1527837790,17,1231,0,45,0
3,3848,1527836592,41,1527837084,113,491,0,53,0
4,3800,1527836617,30,1527837008,27,391,0,42,0


In [114]:
for index, row in df2.iterrows():
    session.add(Rider(bike_id = row['bike_id'],
                    start_time = row['start_time'],
                    start_station_id = row['start_station_id'],
                    end_time = row['end_time'],
                    end_station_id = row['end_station_id'],
                    duration = row['duration_sec'],
                    user_type = row['user_type'],
                    member_age = row['member_age'],
                    member_gender = row['member_gender']))

In [121]:
from sqlalchemy import inspect, MetaData

md = MetaData(engine, reflect=True)
stationQ = session.query(Station)
print(md.tables)
print('---------------')

# for _row in stationQ.all():
#     print(_row.station_id, _row.latitude, _row.longitude, _row.station_name, _row.neighborhood)


immutabledict({'rider': Table('rider', MetaData(bind=Engine(sqlite:///GoBike.sqlite)), Column('id', INTEGER(), table=<rider>, primary_key=True, nullable=False), Column('bike_id', INTEGER(), table=<rider>), Column('start_time', INTEGER(), table=<rider>), Column('start_station_id', INTEGER(), table=<rider>), Column('end_time', INTEGER(), table=<rider>), Column('end_station_id', INTEGER(), table=<rider>), Column('duration', INTEGER(), table=<rider>), schema=None), 'station': Table('station', MetaData(bind=Engine(sqlite:///GoBike.sqlite)), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station_id', INTEGER(), table=<station>), Column('station_name', VARCHAR(length=255), table=<station>), Column('latitude', FLOAT(), table=<station>), Column('longitude', FLOAT(), table=<station>), Column('neighborhood', VARCHAR(length=255), table=<station>), Column('zipcode', INTEGER(), table=<station>), schema=None)})
---------------


C:\Users\jjahn\Anaconda3\envs\pythondata36\lib\site-packages\ipykernel_launcher.py:3: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
stationCastro = session.query(Station).filter(Station.zipcode == 94114)

for _row in stationCastro.all():
    print(_row.station_id, _row.latitude, _row.longitude, _row.station_name, _row.neighborhood)

In [30]:
pd.read_sql(session.query(Station).filter(Station.neighborhood == 'China Basin').statement, session.bind)

,id,station_id,station_name,latitude,longitude,neighborhood,zipcode


In [62]:
df = pd.read_sql('SELECT * FROM station', conn)

In [63]:
df

,id,station_id,station_name,latitude,longitude,neighborhood,zipcode


In [120]:
riders = session.query(Rider)
for _row in riders.all():
    print(_row.start_time, _row.end_time, _row.duration, (_row.end_time - _row.start_time))

OperationalError: (sqlite3.OperationalError) no such column: rider.user_type [SQL: 'SELECT rider.id AS rider_id, rider.bike_id AS rider_bike_id, rider.start_time AS rider_start_time, rider.start_station_id AS rider_start_station_id, rider.end_time AS rider_end_time, rider.end_station_id AS rider_end_station_id, rider.duration AS rider_duration, rider.user_type AS rider_user_type, rider.member_age AS rider_member_age, rider.member_gender AS rider_member_gender \nFROM rider'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [119]:
session.rollback()
session.close()